In [1]:
#Exercises for the book: Monte Carlo Methods, 2nd edition
import numpy as np
import time

In [20]:
#Page 29 in the book, calculating pi
size = 1000000
start = time.time()
rand_numbers = np.random.random_sample(size)
result_vector = np.sqrt(1 - np.power(rand_numbers,2))
result = np.sum(result_vector)
pi_approx = result / size
end = time.time()
print("The value of pi/4 is = {}".format(pi_approx))
print("The real value of pi/4 is = {}".format(np.pi/4))
print("The relative error is = {0:.4f}%".format(np.abs(pi_approx-np.pi/4)*100/(np.pi/4)))
print("Elapsed time = {} seconds".format(end-start))

The value of pi/4 is = 0.785614055401036
The real value of pi/4 is = 0.7853981633974483
The relative error is = 0.0275%
Elapsed time = 0.050618648529052734 seconds


In [10]:
#Another approach to the same problem
size = 1000000
start = time.time()
x_sq = np.power(np.random.random_sample(size), 2)
y_sq = np.power(np.random.random_sample(size), 2)
counter = 0
for i in range(size):
    if x_sq[i] + y_sq[i] <=1:
        counter += 1
result = counter/size
end = time.time()
print("The value of pi/4 is = {}".format(result))
print("The real value of pi/4 is = {}".format(np.pi/4))
print("The relative error is = {0:.4f}%".format(np.abs(result-np.pi/4)*100/(np.pi/4)))
print("Elapsed time = {} seconds".format(end-start))

The value of pi/4 is = 0.785426
The real value of pi/4 is = 0.7853981633974483
The relative error is = 0.0035%
Elapsed time = 1.0053730010986328 seconds


float

In [3]:
%%file pi_approx.pyx
"""Same approach as before but with cython"""
cpdef tuple show():
    import numpy as np
    import time
    cdef long counter =0, size = 1000000
    cdef double start, end, result
    start = time.time()
    x_sq = np.power(np.random.random_sample(size),2)
    y_sq = np.power(np.random.random_sample(size), 2)

    for i in range(size):
        if x_sq[i] + y_sq[i] < 1.0:
            counter += 1
    result = <double>counter/<double>size
    end = time.time()
     
    a = "The value of pi/4 is = {}".format(result)
    b = "The real value of pi/4 is = {}".format(np.pi/4)
    c = "The relative error is = {0:.4f}%".format(np.abs(result-np.pi/4)*100/(np.pi/4))
    d = "Elapsed time = {} seconds".format(end-start)
    return a, b, c, d


Overwriting pi_approx.pyx


In [2]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize
setup(
    name='pi_approx',
    ext_modules = cythonize('pi_approx.pyx')
)

Overwriting setup.py


In [1]:
%run setup.py build_ext --inplace

Compiling pi_approx.pyx because it changed.
[1/1] Cythonizing pi_approx.pyx
running build_ext
building 'pi_approx' extension
gcc -pthread -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/eduardo/anaconda3/include/python3.5m -c pi_approx.c -o build/temp.linux-x86_64-3.5/pi_approx.o
gcc -pthread -shared -L/home/eduardo/anaconda3/lib -Wl,-rpath=/home/eduardo/anaconda3/lib,--no-as-needed build/temp.linux-x86_64-3.5/pi_approx.o -L/home/eduardo/anaconda3/lib -lpython3.5m -o /home/eduardo/Desktop/Programming/Python/0-Folders/pi_approx.cpython-35m-x86_64-linux-gnu.so


In [1]:
import pi_approx
#pi_approx.show()
%timeit -n 10 pi_approx.show()
#We obtain 0.34 ms vs 1.00 seconds from pure python

10 loops, best of 3: 343 ms per loop


In [1]:
#Double integral experiment with Python not in the book
#Integral of x + y from  2<x<4 and 2<y<6
def show():
    import time
    import numpy as np
    N = 1000
    sumx = 0
    sumy = 0
    for i in range(N):
        y = (6-2) * np.random.rand() + 2
        for j in range(N):
            x = (4-2) * np.random.rand() + 2
            sumx += x + y
        sumy = (sumy + sumx/N)
        sumx = 0
    result = (6-2) * (4-2) * sumy/N
    return result
print(show())

56.511185278312404


In [2]:
%%file cyt_dint.pyx
cpdef double show():
    import time
    import numpy as np
    cdef long N = 1000, i, j
    cdef double Nd, x, y, a ,b, c, d, sumx = 0, sumy = 0, result
    Nd = <double>N
    a = 2.0
    b = 6.0
    c = 2.0
    d = 4.0
    for i in range(N):
        y = (b-a) * np.random.rand() + a
        for j in range(N):
            x = (d-c) * np.random.rand() + c
            sumx += x + y
        sumy = (sumy + sumx/Nd)
        sumx = 0.0
    result = (b-a) * (d-c) * sumy/Nd
    return result

Overwriting cyt_dint.pyx


In [101]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize
setup(
    name = 'cyt_dint',
    ext_modules = cythonize('cyt_dint.pyx')
)

Overwriting setup.py


In [3]:
%run setup.py build_ext --inplace

Compiling cyt_dint.pyx because it changed.
[1/1] Cythonizing cyt_dint.pyx
running build_ext
building 'cyt_dint' extension
gcc -pthread -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/eduardo/anaconda3/include/python3.5m -c cyt_dint.c -o build/temp.linux-x86_64-3.5/cyt_dint.o
gcc -pthread -shared -L/home/eduardo/anaconda3/lib -Wl,-rpath=/home/eduardo/anaconda3/lib,--no-as-needed build/temp.linux-x86_64-3.5/cyt_dint.o -L/home/eduardo/anaconda3/lib -lpython3.5m -o /home/eduardo/Desktop/Programming/Python/0-Folders/cyt_dint.cpython-35m-x86_64-linux-gnu.so


In [2]:
import cyt_dint
cyt_dint.show()

56.10776444350975

In [3]:
%timeit -n 10 cyt_dint.show()
%timeit -n 10 show()

10 loops, best of 3: 314 ms per loop
10 loops, best of 3: 624 ms per loop
